In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,BatchNormalization

In [ ]:
from tensorflow.keras.datasets import cifar10
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
x_train=x_train/255.0
x_test=x_test/255.0

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [ ]:
inputs=keras.Input(shape=(32,32,3))
x=Conv2D(32,3)(inputs)
x=BatchNormalization()(x)
x=keras.activations.relu(x)
x=MaxPooling2D(2)(x)

x=Conv2D(64,3)(x)
x=BatchNormalization()(x)
x=keras.activations.relu(x)
x=MaxPooling2D(2)(x)

x=Conv2D(128,3)(x)
x=BatchNormalization()(x)
x=keras.activations.relu(x)
x=MaxPooling2D(2)(x)

x=layers.Flatten()(x)

outputs=layers.Dense(10,activation='softmax')(x)
model=keras.Model(inputs=inputs,outputs=outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(3e-4),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=10,verbose=2,batch_size=64,validation_data=(x_val,y_val))

Epoch 1/10
625/625 - 10s - 16ms/step - accuracy: 0.4879 - loss: 1.4422 - val_accuracy: 0.4892 - val_loss: 1.4439
Epoch 2/10
625/625 - 3s - 4ms/step - accuracy: 0.6217 - loss: 1.0888 - val_accuracy: 0.5946 - val_loss: 1.1623
Epoch 3/10
625/625 - 5s - 8ms/step - accuracy: 0.6747 - loss: 0.9419 - val_accuracy: 0.6319 - val_loss: 1.0384
Epoch 4/10
625/625 - 4s - 7ms/step - accuracy: 0.7110 - loss: 0.8462 - val_accuracy: 0.6332 - val_loss: 1.0458
Epoch 5/10
625/625 - 3s - 4ms/step - accuracy: 0.7420 - loss: 0.7656 - val_accuracy: 0.6425 - val_loss: 1.0346
Epoch 6/10
625/625 - 3s - 4ms/step - accuracy: 0.7660 - loss: 0.6982 - val_accuracy: 0.6637 - val_loss: 0.9960
Epoch 7/10
625/625 - 3s - 4ms/step - accuracy: 0.7874 - loss: 0.6393 - val_accuracy: 0.6662 - val_loss: 0.9786
Epoch 8/10
625/625 - 3s - 5ms/step - accuracy: 0.8053 - loss: 0.5902 - val_accuracy: 0.6226 - val_loss: 1.1276
Epoch 9/10
625/625 - 3s - 4ms/step - accuracy: 0.8234 - loss: 0.5420 - val_accuracy: 0.6877 - val_loss: 0.9255

In [ ]:
model.save('/content/drive/MyDrive/pretrained_model/cnn_cifar10.keras')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/pretrained_model/cnn_cifar10.keras'

In [ ]:
pretrained_model=keras.models.load_model('/content/drive/MyDrive/pretrained_model/cnn_cifar10.keras')

ValueError: File not found: filepath=/content/drive/MyDrive/pretrained_model/cnn_cifar10.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
pretrained_model.summary()

NameError: name 'pretrained_model' is not defined

In [ ]:
(x_train1,y_train1),(x_test1,y_test1)=keras.datasets.cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
#Transfer_learning_with_only_output_layer_changed With functional api
base_inputs=pretrained_model.input
base_outputs=pretrained_model.layers[-2].output #output layer se pehle ka part
final_outputs=layers.Dense(100,activation='softmax')(base_outputs)
new_model=keras.Model(inputs=base_inputs,outputs=final_outputs)

In [ ]:
new_model.compile(optimizer=keras.optimizers.Adam(3e-4),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

In [ ]:
new_model.fit(x_train1,y_train1,epochs=10,verbose=2,batch_size=64,validation_data=(x_test1,y_test1))

In [ ]:
#transfer_learning_with_fine_tuning
base_inputs=pretrained_model.input
base_outputs=pretrained_model.layers[-2].output
final_outputs=layers.Dense(100,activation='softmax')(base_outputs)
new_model1=keras.Model(inputs=base_inputs,outputs=final_outputs)
for layer in pretrained_model.layers[-3:]:
  layer.trainable=True

In [ ]:
new_model1.compile(optimizer=keras.optimizers.Adam(3e-4),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

In [ ]:
new_model1.fit(x_train1,y_train1,epochs=10,verbose=2,batch_size=64,validation_data=(x_test1,y_test1))

In [ ]:
#Transfer learning with fine tuning with images of different size
from tensorflow.keras.datasets import fashion_mnist
import tensorflow as tf

(x_train_fash, y_train_fash), (x_test_fash, y_test_fash) = fashion_mnist.load_data()
x_train_fash = tf.image.resize(tf.expand_dims(x_train_fash, axis=-1), (32, 32)) # 28,28 to 32,32
x_train_fash = tf.image.grayscale_to_rgb(x_train_fash)
x_test_fash = tf.image.resize(tf.expand_dims(x_test_fash, axis=-1), (32, 32))
x_test_fash = tf.image.grayscale_to_rgb(x_test_fash)
print(x_train_fash.shape)
print(x_test_fash.shape)

In [ ]:
#transfer_learning_with_fine_tuning for Fashion MNIST
base_inputs=pretrained_model.input
base_outputs=pretrained_model.layers[-2].output
final_outputs=layers.Dense(10,activation='softmax')(base_outputs)
new_model2=keras.Model(inputs=base_inputs,outputs=final_outputs)
for layer in pretrained_model.layers[-3:]:
  layer.trainable=True

In [ ]:
new_model2.compile(optimizer=keras.optimizers.Adam(3e-4),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

In [ ]:
#using pre-trained models from Keras API
base_model=keras.applications.MobileNetV2(weights='imagenet',include_top=False,input_shape=(32,32,3))

In [ ]:
new_model2.fit(x_train_fash,y_train_fash,epochs=10,verbose=2,batch_size=64,validation_data=(x_test_fash,y_test_fash))

In [ ]:
base_model.summary()

In [ ]:
base_inputs=base_model.input
base_output=base_model.output
outputs=layers.GlobalAveragePooling2D()(base_output)
outputs=layers.Dense(10,activation='softmax')(outputs)
new_model3=keras.Model(inputs=base_inputs,outputs=outputs)
for layer in base_model.layers[50:]:
  layer.trainable=True

In [ ]:
new_model3.compile(optimizer=keras.optimizers.Adam(3e-4),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

In [ ]:
new_model3.fit(x_train_fash,y_train_fash,epochs=10,verbose=2,batch_size=64,validation_data=(x_test_fash,y_test_fash))